# Segmenting and Clustering Neighborhoods in Toronto

In [1]:
!conda install -c conda-forge geopy --yes
!conda install lxml --y
!conda install beautifulsoup4 --y
!conda install -c conda-forge geopy --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.2

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.2

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - lxml


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         156 KB

The following packages will be UPDATED:

    certifi: 2019.11.28-py36_0 conda-forge --> 2019.11.28-py36_0
    openssl: 1.1.1d-h516909a_0 conda-forge --> 1.1.1d-h7b6447c_4


certifi-2019.11.28   | 156 KB    | 

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
 
#from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## Explore Dataset 

I will use the Pandas read_html function to scrap the table in the Wikipedia page https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M.

In [22]:
#assign url and use pandas to read html table
wiki_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
headers = ["PostalCode","Borough","Neighborhood"]
data = pd.read_html(wiki_url, header=0)

#display dataframe correctly
data = data[0]
#rename columns
data = data.rename(columns= {'Postcode': 'PostalCode'})

data.head()

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [23]:
#remove all rows where Borough is "Not assigned"
data = data[data.Borough != "Not assigned"]
data.head()

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


I'm using a lambda function to aggregate all the neighborhoods that fall in the same borough.

In [26]:
#group neighborhood with lambda function
data = data.groupby(by=['PostalCode','Borough']).agg(lambda x: ','.join(x))
data.head()

,,Neighbourhood
PostalCode,Borough,
M1B,Scarborough,"Rouge,Malvern"
M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
M1E,Scarborough,"Guildwood,Morningside,West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


To replace all "Not assigned" values inside the "Neighborhood" column, I'm creating a list of indexes of all the "Not assigned" values, and then replacing those values with the Borough value at the given index.

In [28]:
#replace all "Not assigned" values with Borough
# Create a list of indexes of the cells where the value of Neighbourhood is "Not assigned"
idx_to_change = data.loc[data['Neighbourhood'] == "Not assigned"].index

# Iterate the list of indices and set Neighbourhood to the value of Borough at the given index
for i in idx_to_change:
    data.iloc[i, 1] = df_postcodes.iloc[i, 0]
    
data.head()

,,Neighbourhood
PostalCode,Borough,
M1B,Scarborough,"Rouge,Malvern"
M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
M1E,Scarborough,"Guildwood,Morningside,West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


In [30]:
#show shape of the final dataframe
data.shape

(103, 1)